In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
# 卷积层参数：ci*co*k^2
# 卷积层后的第一个全连接层参数：co*w*h*out_dim

In [3]:
def nin_block(in_channels, out_channels, kernel_size, strides, padding):
    """
    kernel_size和padding都是针对第一个卷积层的参数
    """
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding),
        nn.ReLU(), nn.Conv2d(out_channels, out_channels, kernel_size=1),
        nn.ReLU(), nn.Conv2d(out_channels, out_channels, kernel_size=1),
        nn.ReLU()
    )

In [4]:
# 结构与Alexnet基本一致，只是在原来的每个卷积层后加了两个1*1卷积层
# ? Dropout一般设置在由较大输入层之前？（如全连接层前）
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2d(3, stride=2),
    nin_block(96, 256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2d(3, stride=2),
    nin_block(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2d(3, stride=2),
    nn.Dropout(0.5),
    nin_block(384, 10, kernel_size=3, strides=1, padding=1),
    nn.AdaptiveAvgPool2d((1,1)), 
    nn.Flatten()
)

In [5]:
# 检查输出形状
X = torch.randn(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, 'Output shape:\t', X.shape)

Sequential Output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d Output shape:	 torch.Size([1, 96, 26, 26])
Sequential Output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d Output shape:	 torch.Size([1, 256, 12, 12])
Sequential Output shape:	 torch.Size([1, 384, 12, 12])
MaxPool2d Output shape:	 torch.Size([1, 384, 5, 5])
Dropout Output shape:	 torch.Size([1, 384, 5, 5])
Sequential Output shape:	 torch.Size([1, 10, 5, 5])
AdaptiveAvgPool2d Output shape:	 torch.Size([1, 10, 1, 1])
Flatten Output shape:	 torch.Size([1, 10])


In [ ]:
from my_model import my_train

In [ ]:
lr, num_epochs, batch_size = 0.001, 10, 128
train_iter, test_iter = train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size,resize=224)   
my_train(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu(), stratergy="Adam")